C:\Users\avcamp9\AppData\Local\Continuum\anaconda3\envs\tf-last\Lib\site-packages\keras_retinanet

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

import os
import re
import sys
import csv
import cv2
import time
import random
import pydicom
import argparse
import openpyxl
import matplotlib
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt


from PIL import Image
from skimage import filters
from skimage.measure import regionprops,label

## Define example paths

In [2]:
## Check all for correct input and output directories and names
inputdatadir = 'D:\\syntheticCasesReady' 
outputdatadir = 'E:\\Preprocessed\\TrainCalcClusterSynthetic'
input_file = '220414_CEmammo_classification_simulationsAstrid.xlsx'

In [3]:
wb_patientfull = openpyxl.load_workbook(os.path.join(inputdatadir, input_file))
sheet_train = wb_patientfull['AllCases']

In [4]:
## For training set
list_train = []
list_breast = []
list_label = []
for cell_obj in sheet_train['A'][1:] :
    list_train.append(cell_obj.value)
for cell_obj in sheet_train['B'][1:] :
    list_breast.append(cell_obj.value)
for cell_obj in sheet_train['F'][1:] :
    list_label.append(cell_obj.value)
  
## For test set
# list_train = []
# list_breast = []
# list_label = []
# for cell_obj in sheet_test['A'][1:] :
#     list_train.append(cell_obj.value)
# for cell_obj in sheet_test['B'][1:] :
#     list_breast.append(cell_obj.value)
# for cell_obj in sheet_test['F'][1:] :
#     list_label.append(cell_obj.value)

# list_label[:] = ['0' if 'ben' in x else x for x in list_label]
# list_label[:] = ['1' if 'mal' in x else x for x in list_label]

list_label = ['0' if x=='benign' else x for x in list_label]
list_label = ['1' if x=='malignant' else x for x in list_label]

In [5]:
output_annotations = 'annotations_train_calccluster_synthetic.csv'
annot_file = open(os.path.join(outputdatadir,output_annotations), 'w')
csv_writer = csv.writer(annot_file)

init_row = ['filename', 'x1', 'x2', 'y1', 'y2', 'labels', 'path_mask'] 
csv_writer.writerow(init_row)

39

## Functions for preprocessing

In [6]:
## 6

def resample_intensities(orig_img, bin_nr=256):
    
    v_count=0
    img_list=[]
    filtered = orig_img.copy()
    
    # Ensure all intensity values are positive
    if np.min(orig_img.flatten()) < 0:
        filtered += np.min(orig_img.flatten())
        
    # Compute resampling step based on min and max values
    resampled = np.zeros_like(filtered)
    max_val_img = np.max(filtered.flatten())
    min_val_img = np.min(filtered.flatten())
    step = (max_val_img-min_val_img) / bin_nr

    # Resample intensity values
    for st in np.arange(step+min_val_img, max_val_img+step, step):
        resampled[(filtered<=st) & (filtered>=st-step)] = v_count
        v_count += 1
    
    return np.array(resampled, dtype=np.uint16)

In [7]:
## 5

def pre_processing_for_img(img):
    temp_img = img.copy()
    
    # Compute thresholds from quantile
    low_thr = np.quantile(temp_img[temp_img>0], 0.01)
    high_thr = np.quantile(temp_img[temp_img>0], 0.99)
    # Set values below low_thr or above high_thr to thr value respectively
    temp_img[temp_img<low_thr] = low_thr
    temp_img[temp_img>high_thr] = high_thr
    
    # If too many unique values, resample intensity values
    if len(np.unique(temp_img[temp_img>0])) > 256:
        temp_img_sampled = resample_intensities(temp_img[temp_img>0])
        temp_img[temp_img>0] = temp_img_sampled    
    # If not too many unique values, rescale based on min and max
    else:
        new_img = (temp_img-np.min(temp_img))/(np.max(temp_img)-np.min(temp_img)) 
        temp_img = (new_img*255).astype(np.uint8)   
    return temp_img

In [8]:
## 4

def crop_img(img_read_re, img_read_le):
    
#     # Apply otsu filter for foreground-background
#     otsu = sitk.OtsuThresholdImageFilter()
#     otsu_image = otsu.Execute(img_read_re)
#     otsu_array = sitk.GetArrayFromImage(otsu_image)
    # Alternative for dicom
    temp_img_le_original = img_read_le.pixel_array
    temp_img_re = img_read_re.pixel_array    

    otsu_val = filters.threshold_otsu(temp_img_re)
    otsu_array = temp_img_re < otsu_val
    
    # Find contours in image
    invert_otsu = (np.ones(otsu_array.shape) - otsu_array).astype(np.uint8)
    (contours,_) = cv2.findContours(invert_otsu, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    max_ctr = contours[0]
    for ctr in contours:
        if cv2.contourArea(ctr) > cv2.contourArea(max_ctr):
            max_ctr = ctr
    img_temp = np.zeros(invert_otsu.shape)
    # Why ctr and not max_ctr? Something special about last ctr in contours? Ah, polygon is never used.
    polygon = ctr
    # Fill contour with ones
    cv2.fillPoly(img_temp, [max_ctr], [1])
    # Image where all outside contours are ones
    otsu_array = np.ones(invert_otsu.shape) - img_temp
        
#     temp_img_re = sitk.GetArrayFromImage(img_read_re)    
#     temp_img_le_original = sitk.GetArrayFromImage(img_read_le)
        
#     # Only img values not in otsu, which means indeed in contour
#     temp_img_re = (np.ones((img_read_re.GetSize()[1],img_read_re.GetSize()[0])) - otsu_array)*temp_img_re
#     temp_img_le = (np.ones((img_read_le.GetSize()[1],img_read_le.GetSize()[0])) - otsu_array)*temp_img_le_original
    # Alternative for dicom
    temp_img_re = (np.ones_like(temp_img_le_original) - otsu_array)*temp_img_re
    temp_img_le = (np.ones_like(temp_img_le_original) - otsu_array)*temp_img_le_original
    
    # Get properties of all labeled regions, e.g. bounding box
    props = regionprops(np.array(temp_img_re>0, np.uint8))
    r0, c0, r1, c1 = props[0].bbox
    
    # Choose subarray in bounding box
    temp_img_re = temp_img_re[r0:r1, c0:c1]   
    # Apply preprocessing function
    temp_img_re = pre_processing_for_img(temp_img_re)
    
    # Choose subarray in bounding box
    temp_img_le = temp_img_le[r0:r1, c0:c1]
    # Apply preprocessing function
    temp_img_le = pre_processing_for_img(temp_img_le)
    
    return temp_img_re, temp_img_le, [r0,r1,c0,c1]

In [9]:
## 3

def load_patient(path_input_recombined, path_input_low_energy):
    
    # Load images of patient
    img_read_re = sitk.ReadImage(path_input_recombined)
    img_read_le = sitk.ReadImage(path_input_low_energy)
    
    # Crop images
    img_re, img_le, box_breast = crop_img(img_read_re, img_read_le)
    
    return img_le,img_re,box_breast

In [10]:
## 3

def load_patient_dicom(path_input_recombined, path_input_low_energy):
    
    # Load images of patient
    img_read_re = pydicom.dcmread(path_input_recombined)
    img_read_le = pydicom.dcmread(path_input_low_energy)
    
    # Crop images
    img_re, img_le, box_breast = crop_img(img_read_re, img_read_le)

    return img_le, img_re, box_breast

### Main preprocessing function

In [11]:
## 2

def preprocessing(path_input_low_energy, path_input_recombined):
    
    # Set up 2 versions of CLAHE filter for histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(16,16))
    clahe_recombined_2 = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(16,16)) 

    # Load preprocessed, cropped images of patient
    img_le, img_re, box_breast = load_patient_dicom(path_input_recombined, path_input_low_energy)
    
    # Apply first version of CLAHE filter to low-energy image
    new_img_le = (img_le-np.min(img_le)) / (np.max(img_le)-np.min(img_le)) 
    im_le = (new_img_le*255).astype(np.uint8)
    x_le = clahe.apply(im_le).astype(np.uint8)
    # Apply 2 versions of CLAHE filter to recombined image
    new_img_re = (img_re-np.min(img_re)) / (np.max(img_re)-np.min(img_re)) 
    new_img_re = (new_img_re*255).astype(np.uint8)
    x_re = clahe.apply(new_img_re).astype(np.uint8)
    x_re_2 = clahe_recombined_2.apply(new_img_re).astype(np.uint8)

    # Merge 3 images
    temp_im_le = Image.fromarray(x_le)
    temp_im_re = Image.fromarray(x_re)
    temp_im_re_2 = Image.fromarray(x_re_2)
    merged_img = Image.fromarray(cv2.merge((x_le,x_re,x_re_2)) )
    # Convert to RGB
    img_rgb = merged_img.convert("RGB")
    img_bgr = np.asarray(img_rgb)[:,:,::-1].copy()
    
    return img_bgr, box_breast

## Prediction: main function

In [12]:
def detect_and_classify_lesions_no_model(path_input_recombined, path_input_low_energy):
    
    img_bgr, box_breast = preprocessing(path_input_low_energy, path_input_recombined)
            
    return img_bgr, box_breast

In [13]:
pixeldict = {sitk.sitkUInt16: 'MET_USHORT'}

reader = sitk.ImageFileReader()     # reader for the input .mha files
reader.SetImageIO("MetaImageIO")

def loc_mask(path_input_struct, path_output_struct, bbox, lesion_label) :
    
    reader.SetFileName(path_input_struct)
    mask_image = reader.Execute()
    mask_array = sitk.GetArrayFromImage(mask_image).astype(np.uint16)
    mask_array = mask_array[bbox[0]:bbox[1],bbox[2]:bbox[3]]
    
    mask_nz = np.nonzero(mask_array)
    if len(mask_nz[0]) > 0 :
        xmin = np.min(mask_nz[0])
        xmax = np.max(mask_nz[0])
        ymin = np.min(mask_nz[1])
        ymax = np.max(mask_nz[1])

        labeled_mask = label(mask_array, connectivity=2)
        print('Unique mask labels', np.unique(labeled_mask))
        if 'ben' in str(lesion_label) or '0' in str(lesion_label) :
            labeled_mask[labeled_mask > 0] = 1
        elif 'mal' in str(lesion_label) or '1' in str(lesion_label) :
            labeled_mask[labeled_mask > 0] = 2
#         matplotlib.image.imsave(path_output_struct, labeled_mask)
        PIL_mask = Image.fromarray(np.uint8(labeled_mask))
        PIL_mask.save(path_output_struct)        
    
        return xmin,xmax,ymin,ymax
    
    else :
        return -1, -1, -1, -1
       

In [14]:
def loc_mask_dicom(path_input_mask, path_output_mask, bbox, lesion_label) :
    
    mask = pydicom.dcmread(path_input_mask, force=True)
    
    mask_array = list(mask[0x4d4d,0x2a00].value)[-7330428:]
    mask_array = np.array(mask_array)
    mask_array = np.reshape(mask_array, (3062,2394))
    mask_array = mask_array[bbox[0]:bbox[1],bbox[2]:bbox[3]]
    
    mask_nz = np.nonzero(mask_array)
    if len(mask_nz[0]) > 0 :
        xmin = np.min(mask_nz[0])
        xmax = np.max(mask_nz[0])
        ymin = np.min(mask_nz[1])
        ymax = np.max(mask_nz[1])

        labeled_mask = label(mask_array, connectivity=2)
        print('Unique mask labels', np.unique(labeled_mask))
        if 'ben' in str(lesion_label) or '0' in str(lesion_label) :
            labeled_mask[labeled_mask > 0] = 1
        elif 'mal' in str(lesion_label) or '1' in str(lesion_label) :
            labeled_mask[labeled_mask > 0] = 2
#         matplotlib.image.imsave(path_output_struct, labeled_mask)
        PIL_mask = Image.fromarray(np.uint8(labeled_mask))
        PIL_mask.save(path_output_mask)        
    
        return xmin,xmax,ymin,ymax
    
    else :
        return -1, -1, -1, -1
       
    

In [19]:
startidx = 190

for idx, patientdir in enumerate(list_train[startidx:]) :
    
    ## Patient and path
    patient = patientdir[patientdir.index('MUMC'):patientdir.index('MUMC')+9]
    breast = patient[-2:]             
    print(idx, startidx+idx, patient)
    
    cc_dir = os.path.join(patientdir, 'CC')
    mlo_dir = os.path.join(patientdir, 'MLO')
    
    ## CC
    
    # read input image and mask
    for file in os.listdir(cc_dir) :
        if 'LOW' in file and not 'mask' in file and '.dcm' in file :
            le_name = file
        elif 'REC' in file and not 'mask' in file and '.dcm' in file :
            rc_name = file    
        elif 'mask' in file and '.dcm' in file :
            mask_name = file
    print(le_name, rc_name, mask_name)
    
    # apply preprocessing to image
    bg_cc, bbox_cc = detect_and_classify_lesions_no_model(os.path.join(cc_dir,le_name), os.path.join(cc_dir,rc_name))
    
    # find bounding box of mask and save if delineated
    output_name_mask = os.path.join(outputdatadir, 'mask_to_png', patient+'_'+breast+'_CC.png')
    x1_cc, x2_cc, y1_cc, y2_cc = loc_mask_dicom(os.path.join(cc_dir,mask_name), output_name_mask, bbox_cc, list_label[idx+startidx])
    
    # Only save if nonzero mask and thus delineated
    if x1_cc > 0 :
        
        # save preprocessed image
        im_cc = Image.fromarray(bg_cc)
        output_name_image = os.path.join(outputdatadir, 'colored_to_jpg', patient+'_'+breast+'_CC.jpg')
        im_cc.save(output_name_image)
        
        # save information in .csv file
        cc_row = [output_name_image, x1_cc, x2_cc, y1_cc, y2_cc, list_label[idx+startidx], output_name_mask]
        csv_writer.writerow(cc_row)
    
    ## MLO
    for file in os.listdir(mlo_dir) :
        if 'LOW' in file and not 'mask' in file and '.dcm' in file :
            le_name = file
        elif 'REC' in file and not 'mask' in file and '.dcm' in file :
            rc_name = file  
        elif 'mask' in file and '.dcm' in file :
            mask_name = file            
    print(le_name, rc_name, mask_name)
    
    # apply preprocessing to image
    bg_mlo, bbox_mlo = detect_and_classify_lesions_no_model(os.path.join(mlo_dir,le_name), os.path.join(mlo_dir,rc_name))
    
    # find bounding box of mask and save if delineated
    output_name_mask = os.path.join(outputdatadir, 'mask_to_png', patient+'_'+breast+'_MLO.png')
    x1_mlo, x2_mlo, y1_mlo, y2_mlo = loc_mask_dicom(os.path.join(mlo_dir,mask_name), output_name_mask, bbox_mlo, list_label[idx+startidx])
    
    # Only save if nonzero mask and thus delineated
    if x1_mlo > 0 :
        
        # save preprocessed image
        im_mlo = Image.fromarray(bg_mlo)
        output_name_image = os.path.join(outputdatadir, 'colored_to_jpg', patient+'_'+breast+'_MLO.jpg')
        im_mlo.save(output_name_image)
        
        # save information in .csv file  
        mlo_row = [output_name_image, x1_mlo, x2_mlo, y1_mlo, y2_mlo, list_label[idx+startidx], output_name_mask]
        csv_writer.writerow(mlo_row)
    

0 190 MUMC_1801
MUMC_1801_CC_L_LOW_ENERGY1.dcm MUMC_1801_CC_L_RECOMBINED1.dcm MUMC_1801_CC_L_LOW_ENERGY1_mask.dcm
Unique mask labels [0 1]
MUMC_1801_MLO_L_LOW_ENERGY1.dcm MUMC_1801_MLO_L_RECOMBINED1.dcm MUMC_1801_MLO_L_LOW_ENERGY1_mask.dcm
Unique mask labels [0 1]
1 191 MUMC_1142
MUMC_1142_CC_R_LOW_ENERGY1_changedDataProvider.dcm MUMC_1142_CC_R_RECOMBINED1.dcm MUMC_1142_CC_R_LOW_ENERGY1_mask.dcm
Unique mask labels [0 1]
MUMC_1142_MLO_R_LOW_ENERGY1_changedDataProvider.dcm MUMC_1142_MLO_R_RECOMBINED1.dcm MUMC_1142_MLO_R_LOW_ENERGY1_mask.dcm
Unique mask labels [0 1]
2 192 MUMC_1126
MUMC_1126_CC_R_LOW_ENERGY1_changedDataProvider.dcm MUMC_1126_CC_R_RECOMBINED1.dcm MUMC_1126_CC_R_LOW_ENERGY1_mask.dcm
Unique mask labels [0 1]
MUMC_1126_MLO_R_LOW_ENERGY1_changedDataProvider.dcm MUMC_1126_MLO_R_RECOMBINED1.dcm MUMC_1126_MLO_R_LOW_ENERGY1_mask.dcm
Unique mask labels [0 1]
3 193 MUMC_1071
MUMC_1071_CC_L_LOW_ENERGY1_changedDataProvider.dcm MUMC_1071_CC_L_RECOMBINED1.dcm MUMC_1071_CC_L_LOW_ENERGY1_

In [20]:
annot_file.close()

In [18]:
patient = 'MUMC_0286'

cc_dir = os.path.join(inputdatadir, patient, 'L_XCCM')
# mlo_dir = os.path.join(inputdatadir, patient, list_breast[idx+startidx]+'_MLO')

## CC

# read input image and mask
for file in os.listdir(cc_dir) :
    if 'LOW' in file and not 'mask' in file and '.dcm' in file :
        le_name = file
    elif 'REC' in file and not 'mask' in file and '.dcm' in file :
        rc_name = file    
    elif 'STRUCT' in file and '.mha' in file :
        mask_name = file
print(le_name, rc_name, mask_name)

# apply preprocessing to image
bg_cc, bbox_cc = detect_and_classify_lesions_no_model(os.path.join(cc_dir,le_name), os.path.join(cc_dir,rc_name))

# find bounding box of mask and save if delineated
output_name_mask = os.path.join(outputdatadir, 'mask_to_png', patient+'_'+list_breast[idx+startidx]+'_CC.png')
x1_cc, x2_cc, y1_cc, y2_cc = loc_mask(os.path.join(cc_dir,mask_name), output_name_mask, bbox_cc, list_label[idx+startidx])

# Only save if nonzero mask and thus delineated
if x1_cc > 0 :

    # save preprocessed image
    im_cc = Image.fromarray(bg_cc)
    output_name_image = os.path.join(outputdatadir, 'colored_to_jpg', patient+'_'+list_breast[idx+startidx]+'_CC.jpg')
    im_cc.save(output_name_image)

    # save information in .csv file
    cc_row = [output_name_image, x1_cc, x2_cc, y1_cc, y2_cc, list_label[idx+startidx], output_name_mask]
    csv_writer.writerow(cc_row)

# ## MLO
# for file in os.listdir(mlo_dir) :
#     if 'LOW' in file and not 'mask' in file and '.dcm' in file :
#         le_name = file
#     elif 'REC' in file and not 'mask' in file and '.dcm' in file :
#         rc_name = file  
#     elif 'STRUCT' in file and '.mha' in file :
#         mask_name = file            
# print(le_name, rc_name, mask_name)

# # apply preprocessing to image
# bg_mlo, bbox_mlo = detect_and_classify_lesions_no_model(os.path.join(mlo_dir,le_name), os.path.join(mlo_dir,rc_name))

# # find bounding box of mask and save if delineated
# output_name_mask = os.path.join(outputdatadir, 'mask_to_png', patient+'_'+list_breast[idx+startidx]+'_MLO.png')
# x1_mlo, x2_mlo, y1_mlo, y2_mlo = loc_mask(os.path.join(mlo_dir,mask_name), output_name_mask, bbox_mlo, list_label[idx+startidx])

# # Only save if nonzero mask and thus delineated
# if x1_mlo > 0 :

#     # save preprocessed image
#     im_mlo = Image.fromarray(bg_mlo)
#     output_name_image = os.path.join(outputdatadir, 'colored_to_jpg', patient+'_'+list_breast[idx+startidx]+'_MLO.jpg')
#     im_mlo.save(output_name_image)

#     # save information in .csv file  
#     mlo_row = [output_name_image, x1_mlo, x2_mlo, y1_mlo, y2_mlo, list_label[idx+startidx], output_name_mask]
#     csv_writer.writerow(mlo_row)


MUMC_0286_XCCM_L_LOW_ENERGY1.dcm MUMC_0286_XCCM_L_RECOMBINED1.dcm MUMC_0286_XCCM_L_STRUCT1.mha
Unique mask labels [0 1]


In [16]:
teststring = 'C:\PhD\syntheticCasesReady\MUMC_0029_R'

In [21]:
testpatient = teststring[teststring.index('MUMC'):teststring.index('MUMC')+9]

In [22]:
testpatient

'MUMC_0029'